In [1]:
import pandas as pd
import pytz
from datetime import timedelta, date, datetime
import time
import matplotlib.pyplot as plt

In [83]:
#Commented code below reads data from all the 62 files and appends them together into a single dataframe.
#Performing indexing & grouping operations on this huge dataframe takes about 1674.39s ~30 min
#results in a very bad performance. A better approach is to read data from each file, perform indexing & grouping
#and then concat
'''
start_time = time.time()

#Initialize an empty dataframe to append the daywise resampled data
dayDF = pd.DataFrame()

filename_constant = "Data/sms-call-internet-mi-{}.txt"
col_list = ['cellID', 'timeInterval', 'countryCode', 'smsIn', 'smsOut', 'callIn', 'callOut', 'internetActivity']

#Generating filenames iteratively
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2013, 11, 1)
end_date = date(2014, 1, 2)
for single_date in daterange(start_date, end_date):
    read_data = pd.read_csv(filename_constant.format(single_date.strftime("%Y-%m-%d")), sep='\t',header=None, names=col_list, parse_dates=True)
    #Convert timeInterval column which has Unix timestamps to datetime object
    read_data['startTime'] = pd.to_datetime(read_data.timeInterval, unit='ms')
    
    #Drop timeInterval column as it is now redundant
    read_data.drop(columns=['timeInterval','countryCode'], inplace=True)
    
    #Group the columns by startTime and cellID
    #data_grouped = read_data.groupby(['startTime', 'cellID'])['smsIn','smsOut','callIn','callOut','internetActivity'].sum()

    #data_grouped = read_data.groupby([Grouper(key='startTime', freq='D'), 'cellID'])['smsIn','smsOut','callIn','callOut','internetActivity'].sum()
    #Resample it daywise and get the total sum of the values
    #data_resample = data_grouped.resample('D', level=0).sum()
    dayDF = dayDF.append(read_data)
#Set multi-level index on cellID & startTime
dayDF = dayDF.set_index(['cellID','startTime'])  
#now use grouper

dayDF = dayDF.groupby([pd.Grouper(level='cellID'), pd.Grouper(level='startTime', freq='D')]).sum()
print('%3.2f s' %(time.time() - start_time))
'''


1674.39 s


### Read 62 data files that has volumes proportional to telecommunication activities (SMS, Call, Internet) that occurred every 10 min. Aggregate to hourly and daily volumes.

In [2]:
start_time = time.time()

#Initialize an empty dataframe to append the daily and hourly resampled data
dailyGridActivity = pd.DataFrame()
hourlyGridActivity = pd.DataFrame()

filename_constant = "Data/sms-call-internet-mi-{}.txt"
col_list = ['gridID', 'timeInterval', 'countryCode', 'smsIn', 'smsOut', 'callIn', 'callOut', 'internetActivity']

#Generating filenames iteratively
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2013, 11, 1)
end_date = date(2014, 1, 2)
for single_date in daterange(start_date, end_date):
    read_data = pd.read_csv(filename_constant.format(single_date.strftime("%Y-%m-%d")), sep='\t',header=None, names=col_list, parse_dates=True)
    
    #Check for null values (Execute until this point seperately before proceeding ahead)
    #print(filename_constant.format(single_date.strftime("%Y-%m-%d")))
    #print(read_data.isnull().sum())
    
    #Convert timeInterval column which has Epoch timestamps to UTC and then convert to Milan's local timezone
    #tz_localize(None) returns local time format, not "UTC+1:00" format
    read_data['startTime'] = pd.to_datetime(read_data.timeInterval, unit='ms', utc=True).dt.tz_convert('CET').dt.tz_localize(None)
    
    #Drop timeInterval columns
    read_data.drop(columns=['timeInterval','countryCode'], inplace=True)
    
    #Groupby gridID and startTime as well as set index 
    #startTime which is 10 min apart is resampled to daily aggregation 
    read_data_daily = read_data.groupby(['gridID', pd.Grouper(key='startTime', freq='D')]).sum()
    dailyGridActivity = pd.concat([dailyGridActivity,read_data_daily]).groupby(['gridID', 'startTime']).sum()
    
    #Groupby gridID and startTime as well as set index 
    #startTime which is 10 min apart is resampled to hourly aggregation 
    read_data_hourly = read_data.groupby(['gridID', pd.Grouper(key='startTime', freq='H')]).sum()
    hourlyGridActivity = pd.concat([hourlyGridActivity,read_data_hourly]).groupby(['gridID', 'startTime']).sum()
    
print('%3.2f s' %(time.time() - start_time))

515.67 s


### Load the geojson file to map the Grid Id over the city of Milan

In [32]:
import geopandas
import geojsonio
milan = geopandas.read_file('Data/milano-grid.geojson')
milan = milan.to_json()
geojsonio.display(milan)

'http://geojson.io/#id=gist:/a0e26a2d5afe5502da2fd756dd9f7053'

![10000 grids spatially distributed over city of Milan](images/City_of_Milan.png)